# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [26]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [27]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb")
cur = conn.cursor()

In [28]:
 conn.set_session(autocommit=True)

In [29]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [30]:
song_files = get_files('data/song_data')

In [31]:
song_files[1]

'/Users/juliafumbarev/Documents/Python/dataengineer_udacity/data_modeling/assignment1/project-template/data/song_data/A/A/A/TRAAARJ128F9320760.json'

In [32]:
filepath = song_files[1]

In [33]:
df = pd.read_json(filepath, typ='series')
df.head(20)

num_songs                            1
artist_id           AR8ZCNI1187B9A069B
artist_latitude                   None
artist_longitude                  None
artist_location                       
artist_name           Planet P Project
song_id             SOIAZJW12AB01853F1
title                       Pink World
duration                     269.81832
year                              1984
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [34]:
col = ['song_id','title','artist_id','year','duration']
song_data = df[col].values
song_data

array(['SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984,
       269.81832], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [35]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [36]:
col = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
artist_data = df[col].values
artist_data

array(['AR8ZCNI1187B9A069B', 'Planet P Project', '', None, None],
      dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [37]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [38]:
log_files = get_files('data/log_data')

In [39]:
filepath = log_files[1]

In [40]:
df = pd.read_json(filepath, lines=True)
df.head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Your Rocky Spine,200,1542931645796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Soziedad Alkoholika,Logged In,Kevin,M,1,Arellano,204.74730,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Va Bien,200,1542931860796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,Franz Ferdinand,Logged In,Kevin,M,2,Arellano,172.01587,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Eleanor Put Your Boots On,200,1542932064796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,Modest Mouse,Logged In,Kevin,M,3,Arellano,209.52771,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Float On,200,1542932236796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
4,Adam Lambert,Logged In,Kevin,M,4,Arellano,266.44853,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Aftermath,200,1542932445796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Parni Valjak,Logged In,Kevin,M,5,Arellano,259.83955,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Dok si pored mene,200,1542932711796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
6,None,Logged In,Kevin,M,6,Arellano,NaN,free,"Harrisburg-Carlisle, PA",GET,Home,1.540007e+12,815,None,200,1542932752796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
7,Flaw,Logged In,Kevin,M,7,Arellano,265.89995,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Not Enough,200,1542932970796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
8,Foo Fighters,Logged In,Kevin,M,8,Arellano,250.14812,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Everlong,200,1542933235796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
9,ÃÂngeles del Infierno,Logged In,Kevin,M,9,Arellano,267.25832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Si TÃÂº No EstÃÂ¡s AquÃÂ­,200,1542933485796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [41]:
df = df[df.page=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Your Rocky Spine,200,1542931645796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Soziedad Alkoholika,Logged In,Kevin,M,1,Arellano,204.74730,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Va Bien,200,1542931860796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,Franz Ferdinand,Logged In,Kevin,M,2,Arellano,172.01587,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Eleanor Put Your Boots On,200,1542932064796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,Modest Mouse,Logged In,Kevin,M,3,Arellano,209.52771,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Float On,200,1542932236796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
4,Adam Lambert,Logged In,Kevin,M,4,Arellano,266.44853,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Aftermath,200,1542932445796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


In [42]:
t = df.ts
t.head()

0    1542931645796
1    1542931860796
2    1542932064796
3    1542932236796
4    1542932445796
Name: ts, dtype: int64

In [43]:
time_data = pd.to_datetime(t, unit='ms')
time_data.head()

0   2018-11-23 00:07:25.796
1   2018-11-23 00:11:00.796
2   2018-11-23 00:14:24.796
3   2018-11-23 00:17:16.796
4   2018-11-23 00:20:45.796
Name: ts, dtype: datetime64[ns]

In [44]:
import datetime as dt
time_df = pd.DataFrame(time_data)
time_df['hour'] = time_df.ts.dt.hour 
time_df['day'] = time_df.ts.dt.day 
time_df['week'] = time_df.ts.dt.isocalendar().week
time_df['month'] = time_df.ts.dt.month 
time_df['year'] = time_df.ts.dt.year 
time_df['weekday'] = time_df.ts.dt.weekday 
time_df.head()

,ts,hour,day,week,month,year,weekday
0,2018-11-23 00:07:25.796,0,23,47,11,2018,4
1,2018-11-23 00:11:00.796,0,23,47,11,2018,4
2,2018-11-23 00:14:24.796,0,23,47,11,2018,4
3,2018-11-23 00:17:16.796,0,23,47,11,2018,4
4,2018-11-23 00:20:45.796,0,23,47,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [45]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [46]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [47]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [48]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
%load_ext sql
%sql postgresql://127.0.0.1/sparkifydb

In [ ]:
%sql select * from songplays where song_id is not null and artist_id is not null;

In [49]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, str(row.length)))
    results = cur.fetchone()
    print(results)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

None
[1542931645796, '66', 'free', None, None, 815, 'Harrisburg-Carlisle, PA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
None
[1542931860796, '66', 'free', None, None, 815, 'Harrisburg-Carlisle, PA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
None
[1542932064796, '66', 'free', None, None, 815, 'Harrisburg-Carlisle, PA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
None
[1542932236796, '66', 'free', None, None, 815, 'Harrisburg-Carlisle, PA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
None
[1542932445796, '66', 'free', None, None, 815, 'Harrisburg-Carlisle, PA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1

None
[1542992917796, '58', 'paid', None, None, 852, 'Augusta-Richmond County, GA-SC', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
None
[1542992997796, '88', 'paid', None, None, 812, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
None
[1542993192796, '88', 'paid', None, None, 812, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
None
[1542993212796, '32', 'free', None, None, 885, 'New York-Newark-Jersey City, NY-NJ-PA', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
None
[1542993250796, '58', 'paid', None, None, 852, 'Augusta-Richmond County, GA-SC', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.